### Importing Required Libraries

In [2]:

# Real-time face emotion detection using OpenCV
import cv2
import numpy as np
from tensorflow.keras.models import load_model

#

### Real Time Face Detection

In [4]:

# Load pre-trained model and required files
model = load_model('./face_emotion.h5')  #  model path
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Initialize face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to gray scale
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract the region of interest (face) and preprocess for the model
        roi_color = frame[y:y + h, x:x + w]
        roi_resized = cv2.resize(roi_color, (224, 224))  # Resize Image dimenssion into 224x224
        
        # Normalize pixel values to [0, 1]
        roi_normalized = roi_resized / 255.0

        # Add batch and channel dimensions
        roi_expanded = np.expand_dims(roi_normalized, axis=0)

        # Predict emotion
        predictions = model.predict(roi_expanded)
        max_index = np.argmax(predictions[0])
        predicted_emotion = emotion_labels[max_index]

        # Draw rectangle around the face and display the predicted emotion
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Show the frame with detections
    cv2.imshow('Real-Time Face Emotion Detection', frame)

    # Break loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 124ms/step
